In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
from candlestick_screener.patterns import patterns
import talib
from flask import request

In [2]:
ticker = yf.Ticker('MSFT')

hist = ticker.history(period='7d', interval='1m')

df = pd.DataFrame(hist)
df.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-09-22 09:30:00-04:00,296.725006,296.869995,295.929993,296.595001,1067177,0,0
2021-09-22 09:31:00-04:00,296.579987,296.915009,296.059998,296.179993,103604,0,0
2021-09-22 09:32:00-04:00,296.140015,296.158295,295.761505,295.910004,68212,0,0
2021-09-22 09:33:00-04:00,296.109985,296.109985,295.779999,295.820007,103321,0,0
2021-09-22 09:34:00-04:00,295.500000,295.789886,295.299988,295.309998,106053,0,0


In [3]:
'2021-09-22 09:30:00-04:00' in df.index

True

In [4]:
df = df.drop(['Dividends', 'Stock Splits'], axis=1)
df.head()

,Open,High,Low,Close,Volume
Datetime,,,,,
2021-09-22 09:30:00-04:00,296.725006,296.869995,295.929993,296.595001,1067177
2021-09-22 09:31:00-04:00,296.579987,296.915009,296.059998,296.179993,103604
2021-09-22 09:32:00-04:00,296.140015,296.158295,295.761505,295.910004,68212
2021-09-22 09:33:00-04:00,296.109985,296.109985,295.779999,295.820007,103321
2021-09-22 09:34:00-04:00,295.500000,295.789886,295.299988,295.309998,106053


This is where I found the code for calculating the Average True Range (ATR)

https://www.learnpythonwithrune.org/calculate-the-average-true-range-atr-easy-with-pandas-dataframes/

In [5]:
# high_low = df['High'] - df['Low']
# high_close = np.abs(df['High'] - df['Close'].shift())
# low_close = np.abs(df['Low'] - df['Close'].shift())

# ranges = pd.concat([high_low, high_close, low_close], axis=1)
# true_range = np.max(ranges, axis=1)

# window = 14

# atr = true_range.rolling(window).sum() / window

In [6]:
# df['ATR'] = atr
# df.head(20)

In [7]:

pattern_function = getattr(talib, 'CDLDOJI')
symbol = 'MSFT'

In [8]:
result = pattern_function(df['Open'], df['High'], df['Low'], df['Close'])
df['DOJI'] = result
last = result.tail(1).values[0]
df


,Open,High,Low,Close,Volume,DOJI
Datetime,,,,,,
2021-09-22 09:30:00-04:00,296.725006,296.869995,295.929993,296.595001,1067177,0
2021-09-22 09:31:00-04:00,296.579987,296.915009,296.059998,296.179993,103604,0
2021-09-22 09:32:00-04:00,296.140015,296.158295,295.761505,295.910004,68212,0
2021-09-22 09:33:00-04:00,296.109985,296.109985,295.779999,295.820007,103321,0
2021-09-22 09:34:00-04:00,295.500000,295.789886,295.299988,295.309998,106053,0
...,...,...,...,...,...,...
2021-09-30 13:38:00-04:00,283.212006,283.440002,283.197510,283.429993,36200,0
2021-09-30 13:39:00-04:00,283.459991,283.459991,283.285004,283.290009,52962,0
2021-09-30 13:40:00-04:00,283.290009,283.399994,283.230011,283.299988,40060,100


In [11]:
atr_function = getattr(talib, 'ATR')

atr_result = atr_function(df['High'], df['Low'], df['Close'], timeperiod=14)

atr_factor = 2.5

df['Trailing Stop'] = df['Close'] - (atr_result * atr_factor)

In [13]:
df['Long Entry'] = np.nan

for index, row in df.iterrows():
    if row['DOJI'] > 0:
        df.loc[index, 'Long Entry'] = 1
    else:
        df.loc[index, 'Long Entry'] = 0

In [16]:
df['Pct Change'] = df['Close'].pct_change()

In [18]:
df['Initialize Stop Loss'] = np.where(df['Long Entry']==1,df['Trailing Stop'],0.0)

In [27]:
df['Recalculate Stop Loss Once in Trade'] = np.nan

for index, row in df.iterrows():
    if row['Initialize Stop Loss'] > 0.0:
        df.loc[index, 'Recalculate Stop Loss Once in Trade'] = row['Initialize Stop Loss']

    elif row['Initialize Stop Loss'] > 0.0 or row['Pct Change'] > 0.0:
        if row['Trailing Stop'] > row['Recalculate Stop Loss Once in Trade'].shift(1):
            df.loc[index,'Recalculate Stop Loss Once in Trade'] = row['Trailing Stop']
        else:
            df.loc[index,'Recalculate Stop Loss Once in Trade'] = df.loc[index, 'Recalculate Stop Loss Once in Trade'].shift(1)
    
    else:
        df.loc[index,'Recalculate Stop Loss Once in Trade'] = row['Recalculate Stop Loss Once in Trade'].shift(1)

TypeError: tuple indices must be integers or slices, not str

In [19]:
df.tail(40)

,Open,High,Low,Close,Volume,DOJI,Trailing Stop,Long Entry,Pct Change,Initialize Stop Loss
Datetime,,,,,,,,,,
2021-09-30 13:03:00-04:00,282.463501,282.516388,282.360992,282.429993,35365,0,281.766409,0.0,-0.000120,0.000000
2021-09-30 13:04:00-04:00,282.380005,282.750000,282.369995,282.725006,63242,0,282.040963,0.0,0.001045,0.000000
2021-09-30 13:05:00-04:00,282.730011,282.809998,282.670105,282.721008,53489,100,282.060845,1.0,-0.000014,282.060845
2021-09-30 13:06:00-04:00,282.720001,282.720001,282.535004,282.589996,39566,0,281.943772,0.0,-0.000463,0.000000
2021-09-30 13:07:00-04:00,282.600006,282.829987,282.570007,282.739990,43817,0,282.093500,0.0,0.000531,0.000000
2021-09-30 13:08:00-04:00,282.739990,283.220001,282.700012,283.220001,69160,0,282.526834,0.0,0.001698,0.000000
2021-09-30 13:09:00-04:00,283.250000,283.329987,283.179993,283.309998,66074,0,282.639557,0.0,0.000318,0.000000
2021-09-30 13:10:00-04:00,283.315002,283.519989,283.309998,283.440002,72910,0,282.779952,0.0,0.000459,0.000000
2021-09-30 13:11:00-04:00,283.410004,283.429993,283.279999,283.369995,41626,0,282.728519,0.0,-0.000247,0.000000


In [ ]:




# df['Stop Loss'] =

# atr_multiplier = 2.5
# previous_close = 0
# previous_stop = 0

# for index, row in df.iterrows():
#     if row['DOJI'] > 0:
#         df.loc[index, 'Long'] = 1
#         df.loc[index, "Stop Loss"] = row['Close'] - (row['ATR'] * atr_multiplier) 
          
#     elif row['Close'] >= previous_close:
#         df.loc[index, 'Long'] = 1
#         df.loc[index, "Stop Loss"] = row['Close'] - (row['ATR'] * atr_multiplier) 
    
#     elif row['Close'] <= previous_close:
#         df.loc[index, 'Long'] = 1
#         df.loc[index, "Stop Loss"] = previous_stop

#     elif row['Low'] >= previous_stop:
#         df.loc[index, 'Long'] = -1
#         df.loc[index, "Stop Loss"] = row['Close'] - (row['ATR'] * atr_multiplier)

#     else:
#         df.loc[index, 'Long'] = 0

         

    
#     previous_stop = row['Close'] - (row['ATR'] * atr_multiplier)
#     previous_close = row['Close']

df.tail(40)

In [ ]:
equity_balance = 20000
expected_return = 1.6
prob_win = 0.7
prob_loss = 1 - prob_win

pct_equity = (expected_return * prob_win - prob_loss) / expected_return
full_kelly = pct_equity * equity_balance
fractional_kelly = 0.33 * full_kelly

fractional_kelly


In [ ]:
df['Exit'] = df['Close'] + df['Dynamic Stop Loss']

In [ ]:
df.dropna()

In [ ]:
df['Equity Balance'] = equity_balance


In [ ]:
import hvplot.pandas

# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["SMA50", "SMA200"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot